In [1]:
# Notebook: Regime-aware coin policy (v1) – Fear ON, Greed/Neutral OFF
# Author: Aryan Nayak
# Date: 2025-08-18
import sys, platform, numpy as np, pandas as pd
np.random.seed(42)
print("Python:", sys.version)
print("Pandas:", pd.__version__)
print("Platform:", platform.platform())


Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Pandas: 2.2.2
Platform: Linux-6.1.123+-x86_64-with-glibc2.35


In [1]:
required = ['coin_day_fg','mkt_day_fg','daily_baseline','rules']
missing = [r for r in required if r not in globals()]
print("Missing:", missing)


Missing: ['coin_day_fg', 'mkt_day_fg', 'daily_baseline', 'rules']


In [2]:
import pandas as pd, os

if any(x in ['coin_day_fg','mkt_day_fg'] for x in missing):
    DATA_DIR = '/content/drive/MyDrive/Crypto/processed'  # change if needed
    coin_path = os.path.join(DATA_DIR, 'coin_day_with_fg.csv')
    mkt_path  = os.path.join(DATA_DIR, 'market_day_with_fg.csv')
    if not (os.path.exists(coin_path) and os.path.exists(mkt_path)):
        raise FileNotFoundError("Missing coin_day_with_fg.csv or market_day_with_fg.csv. Update DATA_DIR or rebuild aggregates.")
    coin_day_fg = pd.read_csv(coin_path, parse_dates=['date_utc'])
    mkt_day_fg  = pd.read_csv(mkt_path,  parse_dates=['date_utc'])
    print("Loaded coin_day_fg and mkt_day_fg from:", DATA_DIR)
else:
    print("coin_day_fg and mkt_day_fg already in memory.")


Loaded coin_day_fg and mkt_day_fg from: /content/drive/MyDrive/Crypto/processed


In [3]:
import json, pathlib

if 'rules' in missing:
    POLICY_PATH = pathlib.Path('/content/drive/MyDrive/Crypto/processed/policy_v1.json')
    if POLICY_PATH.exists():
        with open(POLICY_PATH) as f:
            policy = json.load(f)
        rules = {
            'Greed':  {'favor_coins': set(policy['rules']['Greed']['favor_coins']),
                       'avoid_coins': set(policy['rules']['Greed']['avoid_coins'])},
            'Fear':   {'favor_coins': set(policy['rules']['Fear']['favor_coins']),
                       'avoid_coins': set(policy['rules']['Fear']['avoid_coins'])},
            'Neutral':{'favor_coins': set(policy['rules']['Neutral']['favor_coins']),
                       'avoid_coins': set(policy['rules']['Neutral']['avoid_coins'])},
        }
        print("Loaded rules from policy_v1.json")
    else:
        rules = {
            'Greed': {
                'favor_coins': set([]),
                'avoid_coins': set(['ATOM','DOT','IO','LDO','POPCAT','SEI','STRK','TRUMP','XRP','ZK'])
            },
            'Fear': {
                'favor_coins': set(['AIXBT','BTC','ENA','ETH','HYPE','JELLY','MELANIA','SOL','SUI','TRUMP']),
                'avoid_coins': set(['@107','@4','@8','ADA','BNB','DYDX','FARTCOIN','JUP','PURR/USDC','TNSR'])
            },
            'Neutral': {
                'favor_coins': set([]),
                'avoid_coins': set([])
            }
        }
        print("Defined rules inline (policy_v1.json not found).")
else:
    print("rules already in memory.")


Loaded rules from policy_v1.json


In [4]:
def label_coin_policy_q(row):
    regime, c = row['sentiment_binary'], row['coin']
    if regime == 'Greed':
        if c in rules['Greed']['avoid_coins']: return 'avoid'
        return 'neutral'
    if regime == 'Fear':
        if c in rules['Fear']['favor_coins']: return 'favor'
        if c in rules['Fear']['avoid_coins']: return 'avoid'
        return 'neutral'
    return 'neutral'

coin_day_fg['policy_q'] = coin_day_fg.apply(label_coin_policy_q, axis=1)
coin_day_fg.groupby(['sentiment_binary','policy_q']).size().reset_index(name='rows').sort_values(['sentiment_binary','rows'], ascending=[True, False])


,sentiment_binary,policy_q,rows
2,Fear,neutral,645
1,Fear,favor,385
0,Fear,avoid,213
4,Greed,neutral,2879
3,Greed,avoid,228
5,Neutral,neutral,705


In [5]:
if 'daily_baseline' in missing:
    daily_baseline = (
        coin_day_fg.groupby('date_utc', as_index=False)
        .agg(baseline_pnl=('pnl_net_sum','sum'))
        .merge(mkt_day_fg[['date_utc','sentiment_binary','value']], on='date_utc', how='left')
    )
    print("Built daily_baseline.")
else:
    print("daily_baseline already in memory.")


Built daily_baseline.


In [6]:
# Weights (v1)
regime_weights_v5 = {
    'Greed':  {'favor': 0.0,  'neutral': 0.0,  'avoid': 0.0},
    'Fear':   {'favor': 1.0,  'neutral': 0.50, 'avoid': 0.0},
    'Neutral':{'favor': 0.0,  'neutral': 0.0,  'avoid': 0.0},
}

def row_weight_v5(r):
    rw = regime_weights_v5.get(r['sentiment_binary'], {'favor': 0.0, 'neutral': 0.0, 'avoid': 0.0})
    return rw.get(r['policy_q'], 0.0)

# Per-day guard scaffolding (kept; Greed is off so guard is inert)
day_mode = (
    coin_day_fg.groupby('date_utc', as_index=False)
    .agg(day_sentiment=('sentiment_binary','first'))
)

greed_days = day_mode.loc[day_mode['day_sentiment']=='Greed', 'date_utc']
per_day_median = (
    coin_day_fg[coin_day_fg['date_utc'].isin(greed_days)]
    .groupby('date_utc', as_index=False)['size_usd_sum'].median()
    .rename(columns={'size_usd_sum':'day_median_size'})
)

coin_day_guarded = (
    coin_day_fg.merge(day_mode, on='date_utc', how='left')
               .merge(per_day_median, on='date_utc', how='left')
)

mask_greed = coin_day_guarded['day_sentiment'].eq('Greed')
mask_keep = (~mask_greed) | (coin_day_guarded['size_usd_sum'] >= coin_day_guarded['day_median_size'].fillna(-1))
coin_day_guarded = coin_day_guarded.loc[mask_keep].copy()

coin_day_guarded['weight_v5'] = coin_day_guarded.apply(row_weight_v5, axis=1)
coin_day_guarded['strategy_contrib'] = coin_day_guarded['pnl_net_sum'] * coin_day_guarded['weight_v5']

daily_weighted_v5 = (
    coin_day_guarded.groupby('date_utc', as_index=False)['strategy_contrib'].sum()
    .rename(columns={'strategy_contrib':'strategy_pnl_v5'})
)

daily_compare_v5 = daily_baseline.merge(daily_weighted_v5, on='date_utc', how='left')
daily_compare_v5['strategy_pnl_v5'] = daily_compare_v5['strategy_pnl_v5'].fillna(0.0)
daily_compare_v5['improvement_v5'] = daily_compare_v5['strategy_pnl_v5'] - daily_compare_v5['baseline_pnl']

compare_stats_v5 = (
    daily_compare_v5.groupby('sentiment_binary', dropna=False)
    .agg(
        days=('date_utc','count'),
        baseline_mean=('baseline_pnl','mean'),
        strategy_mean=('strategy_pnl_v5','mean'),
        improvement_mean=('improvement_v5','mean'),
        improvement_sum=('improvement_v5','sum')
    )
    .reset_index()
    .sort_values('strategy_mean', ascending=False)
)

compare_stats_v5


,sentiment_binary,days,baseline_mean,strategy_mean,improvement_mean,improvement_sum
0,Fear,103,37121.306906,39581.989352,2460.682447,2.534503e+05
1,Greed,309,16076.498880,0.000000,-16076.498880,-4.967638e+06
2,Neutral,64,19687.007152,0.000000,-19687.007152,-1.259968e+06


In [7]:
import json
from pathlib import Path

OUT_DIR = Path('/content/drive/MyDrive/Crypto/processed')
OUT_DIR.mkdir(parents=True, exist_ok=True)

daily_compare_v5.to_csv(OUT_DIR / 'daily_compare_v1.csv', index=False)
compare_stats_v5.to_csv(OUT_DIR / 'regime_compare_stats_v1.csv', index=False)

final_policy = {
    'rules': {
        'Greed':  {'favor_coins': [], 'avoid_coins': sorted(list(rules['Greed']['avoid_coins']))},
        'Fear':   {'favor_coins': sorted(list(rules['Fear']['favor_coins'])),
                   'avoid_coins': sorted(list(rules['Fear']['avoid_coins']))},
        'Neutral':{'favor_coins': [], 'avoid_coins': []},
    },
    'weights': regime_weights_v5,
    'liquidity_guard': {'enabled_on_greed': True, 'method': 'per-day median size_usd_sum; keep >= median'},
}

with open(OUT_DIR / 'policy_v1.json', 'w') as f:
    json.dump(final_policy, f, indent=2)

print("Saved to:", OUT_DIR)


Saved to: /content/drive/MyDrive/Crypto/processed


In [8]:
print("Regime summary (v1):")
display(compare_stats_v5)

overall = {
    'baseline_mean_total': float(daily_compare_v5['baseline_pnl'].mean()),
    'strategy_mean_total': float(daily_compare_v5['strategy_pnl_v5'].mean()),
    'improvement_mean_total': float((daily_compare_v5['strategy_pnl_v5'] - daily_compare_v5['baseline_pnl']).mean()),
    'improvement_sum_total': float((daily_compare_v5['strategy_pnl_v5'] - daily_compare_v5['baseline_pnl']).sum()),
}
overall


Regime summary (v1):


,sentiment_binary,days,baseline_mean,strategy_mean,improvement_mean,improvement_sum
0,Fear,103,37121.306906,39581.989352,2460.682447,2.534503e+05
1,Greed,309,16076.498880,0.000000,-16076.498880,-4.967638e+06
2,Neutral,64,19687.007152,0.000000,-19687.007152,-1.259968e+06


{'baseline_mean_total': 21115.758871576196,
 'strategy_mean_total': 8565.010301048453,
 'improvement_mean_total': -12550.748570527745,
 'improvement_sum_total': -5974156.319571206}

In [ ]:
from google.colab import drive
from pathlib import Path

# 1) Mount Drive
drive.mount('/content/drive')

# 2) Define file paths based on your locations
FEAR_GREED_CSV = Path('/content/drive/MyDrive/Crypto/fear_greed_index.csv')
HISTORICAL_CSV = Path('/content/drive/MyDrive/Crypto/historical_data.csv')

# 3) Quick existence check
print("Fear & Greed file exists:", FEAR_GREED_CSV.exists())
print("Historical data file exists:", HISTORICAL_CSV.exists())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fear & Greed file exists: True
Historical data file exists: True


In [ ]:
import pandas as pd

FEAR_GREED_CSV = '/content/drive/MyDrive/Crypto/fear_greed_index.csv'
HISTORICAL_CSV = '/content/drive/MyDrive/Crypto/historical_data.csv'

# Load small samples
hl_sample = pd.read_csv(HISTORICAL_CSV, nrows=5, low_memory=False)
fg_sample = pd.read_csv(FEAR_GREED_CSV, nrows=5, low_memory=False)

print("Hyperliquid sample:")
display(hl_sample)
print("Fear & Greed sample:")
display(fg_sample)

# Peek at dtypes with a slightly larger sniff
hl_head = pd.read_csv(HISTORICAL_CSV, nrows=1000, low_memory=False)
fg_head = pd.read_csv(FEAR_GREED_CSV, nrows=1000, low_memory=False)

print("\nHyperliquid columns:", list(hl_head.columns))
print("Fear & Greed columns:", list(fg_head.columns))

print("\nHyperliquid dtypes:")
print(hl_head.dtypes)
print("\nFear & Greed dtypes:")
print(fg_head.dtypes)


Hyperliquid sample:


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12
3,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9874,142.98,1142.04,BUY,02-12-2024 22:50,1146.558564,Buy,0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050043,1.080000e+15,1.730000e+12
4,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9894,8.73,69.75,BUY,02-12-2024 22:50,1289.488521,Buy,0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.003055,1.050000e+15,1.730000e+12


Fear & Greed sample:


,timestamp,value,classification,date
0,1517463000,30,Fear,2018-02-01
1,1517549400,15,Extreme Fear,2018-02-02
2,1517635800,40,Fear,2018-02-03
3,1517722200,24,Extreme Fear,2018-02-04
4,1517808600,11,Extreme Fear,2018-02-05



Hyperliquid columns: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']
Fear & Greed columns: ['timestamp', 'value', 'classification', 'date']

Hyperliquid dtypes:
Account              object
Coin                 object
Execution Price     float64
Size Tokens         float64
Size USD            float64
Side                 object
Timestamp IST        object
Start Position      float64
Direction            object
Closed PnL          float64
Transaction Hash     object
Order ID              int64
Crossed                bool
Fee                 float64
Trade ID            float64
Timestamp           float64
dtype: object

Fear & Greed dtypes:
timestamp          int64
value              int64
classification    object
date              object
dtype: object


In [ ]:
import pandas as pd
import numpy as np

# Paths
FEAR_GREED_CSV = '/content/drive/MyDrive/Crypto/fear_greed_index.csv'
HISTORICAL_CSV = '/content/drive/MyDrive/Crypto/historical_data.csv'

# 1) Load
hl = pd.read_csv(HISTORICAL_CSV, low_memory=False)
fg = pd.read_csv(FEAR_GREED_CSV, low_memory=False)

# 2) Normalize column names (spaces -> underscores, lowercase)
def normalize_cols(df):
    df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]
    return df

hl = normalize_cols(hl)
fg = normalize_cols(fg)

# 3) Cast numeric columns for Hyperliquid
num_cols_hl = ['execution_price', 'size_tokens', 'size_usd', 'closed_pnl', 'fee', 'trade_id', 'timestamp']
for c in num_cols_hl:
    if c in hl.columns:
        hl[c] = pd.to_numeric(hl[c], errors='coerce')

# Ensure 'crossed' is boolean if present
if 'crossed' in hl.columns and hl['crossed'].dtype != bool:
    # common cases: 'True'/'False' strings or 0/1
    hl['crossed'] = hl['crossed'].map({True: True, False: False, 'True': True, 'False': False, 1: True, 0: False}).fillna(False).astype(bool)

# Standardize some text columns
for c in ['account', 'coin', 'side', 'direction', 'transaction_hash']:
    if c in hl.columns:
        hl[c] = hl[c].astype(str).str.strip()

# 4) Fear & Greed numeric casting
if 'value' in fg.columns:
    fg['value'] = pd.to_numeric(fg['value'], errors='coerce')

print("HL shape:", hl.shape, "FG shape:", fg.shape)
hl.head(3)


HL shape: (211224, 16) FG shape: (2644, 4)


,account,coin,execution_price,size_tokens,size_usd,side,timestamp_ist,start_position,direction,closed_pnl,transaction_hash,order_id,crossed,fee,trade_id,timestamp
0,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9769,986.87,7872.16,BUY,02-12-2024 22:50,0.000000,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.345404,8.950000e+14,1.730000e+12
1,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9800,16.00,127.68,BUY,02-12-2024 22:50,986.524596,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.005600,4.430000e+14,1.730000e+12
2,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,@107,7.9855,144.09,1150.63,BUY,02-12-2024 22:50,1002.518996,Buy,0.0,0xec09451986a1874e3a980418412fcd0201f500c95bac...,52017706630,True,0.050431,6.600000e+14,1.730000e+12


In [ ]:
import pandas as pd
import numpy as np

# Fear & Greed: UNIX seconds -> UTC datetime/date
fg['timestamp_utc'] = pd.to_datetime(fg['timestamp'], unit='s', utc=True)
fg['fg_date_utc'] = fg['timestamp_utc'].dt.date

# Trades: IST string -> UTC datetime/date
ts_naive = pd.to_datetime(hl['timestamp_ist'], format='%d-%m-%Y %H:%M', errors='coerce')
hl['timestamp_ist_dt'] = ts_naive.dt.tz_localize('Asia/Kolkata', nonexistent='NaT', ambiguous='NaT')
hl['timestamp_utc'] = hl['timestamp_ist_dt'].dt.tz_convert('UTC')
hl['date_utc'] = hl['timestamp_utc'].dt.date

print("HL timestamp_utc NaT ratio:", hl['timestamp_utc'].isna().mean())
print("FG timestamp_utc NaT ratio:", fg['timestamp_utc'].isna().mean())

display(hl[['timestamp_ist', 'timestamp_utc']].head(5))
display(fg[['timestamp', 'timestamp_utc', 'fg_date_utc']].head(5))


HL timestamp_utc NaT ratio: 0.0
FG timestamp_utc NaT ratio: 0.0


,timestamp_ist,timestamp_utc
0,02-12-2024 22:50,2024-12-02 17:20:00+00:00
1,02-12-2024 22:50,2024-12-02 17:20:00+00:00
2,02-12-2024 22:50,2024-12-02 17:20:00+00:00
3,02-12-2024 22:50,2024-12-02 17:20:00+00:00
4,02-12-2024 22:50,2024-12-02 17:20:00+00:00


,timestamp,timestamp_utc,fg_date_utc
0,1517463000,2018-02-01 05:30:00+00:00,2018-02-01
1,1517549400,2018-02-02 05:30:00+00:00,2018-02-02
2,1517635800,2018-02-03 05:30:00+00:00,2018-02-03
3,1517722200,2018-02-04 05:30:00+00:00,2018-02-04
4,1517808600,2018-02-05 05:30:00+00:00,2018-02-05


In [ ]:
# PnL fields
hl['pnl_gross'] = hl['closed_pnl']
hl['pnl_net'] = hl['closed_pnl'] - hl['fee'].fillna(0)

# Return per trade
hl['return_per_trade'] = np.where(hl['size_usd'].fillna(0) != 0, hl['pnl_net'] / hl['size_usd'], np.nan)

# Account-day
acc_day = (
    hl.groupby(['account', 'date_utc'])
      .agg(pnl_net_sum=('pnl_net','sum'),
           pnl_net_mean=('pnl_net','mean'),
           pnl_gross_sum=('pnl_gross','sum'),
           return_mean=('return_per_trade','mean'),
           size_usd_sum=('size_usd','sum'),
           size_usd_mean=('size_usd','mean'),
           trades_count=('pnl_net','count'),
           win_rate=('pnl_net', lambda s: (s>0).mean()))
      .reset_index()
)

# Market-day
mkt_day = (
    hl.groupby('date_utc')
      .agg(pnl_net_sum=('pnl_net','sum'),
           pnl_net_mean=('pnl_net','mean'),
           pnl_gross_sum=('pnl_gross','sum'),
           return_mean=('return_per_trade','mean'),
           size_usd_sum=('size_usd','sum'),
           size_usd_mean=('size_usd','mean'),
           trades_count=('pnl_net','count'),
           win_rate=('pnl_net', lambda s: (s>0).mean()))
      .reset_index()
)

# Coin-day (optional now)
coin_day = (
    hl.groupby(['coin','date_utc'])
      .agg(pnl_net_sum=('pnl_net','sum'),
           pnl_net_mean=('pnl_net','mean'),
           return_mean=('return_per_trade','mean'),
           size_usd_sum=('size_usd','sum'),
           size_usd_mean=('size_usd','mean'),
           trades_count=('pnl_net','count'),
           win_rate=('pnl_net', lambda s: (s>0).mean()))
      .reset_index()
)

acc_day.head(3), mkt_day.head(3), coin_day.head(3)


(                                      account    date_utc  pnl_net_sum  \
 0  0x083384f897ee0f19899168e3b1bec365f52a9012  2024-11-11  -167.796055   
 1  0x083384f897ee0f19899168e3b1bec365f52a9012  2024-11-17   -67.883615   
 2  0x083384f897ee0f19899168e3b1bec365f52a9012  2024-11-18   -94.937983   
 
    pnl_net_mean  pnl_gross_sum  return_mean  size_usd_sum  size_usd_mean  \
 0     -0.948000            0.0    -0.000123     900880.13    5089.718249   
 1     -0.998288            0.0    -0.000115     542413.18    7976.664412   
 2     -2.373450            0.0    -0.000100     949380.00   23734.500000   
 
    trades_count  win_rate  
 0           177       0.0  
 1            68       0.0  
 2            40       0.0  ,
      date_utc  pnl_net_sum  pnl_net_mean  pnl_gross_sum  return_mean  \
 0  2023-04-30     0.000000      0.000000       0.000000     0.000000   
 1  2023-12-04   -12.501455     -1.389051       0.000000    -0.000250   
 2  2023-12-13  -294.499239    -98.166413    -281.80

In [ ]:
# Reduce F&G to one row per date (keep value + classification)
fg_small = (
    fg[['fg_date_utc', 'value', 'classification']]
    .drop_duplicates(subset=['fg_date_utc'])
)

# Binary label helper
def to_binary_label(x):
    x = str(x).lower()
    if 'greed' in x: return 'Greed'
    if 'fear'  in x: return 'Fear'
    if 'neutral' in x: return 'Neutral'
    return np.nan

fg_small['sentiment_binary'] = fg_small['classification'].map(to_binary_label)

# Join
acc_day_fg = acc_day.merge(fg_small, left_on='date_utc', right_on='fg_date_utc', how='left')
mkt_day_fg = mkt_day.merge(fg_small, left_on='date_utc', right_on='fg_date_utc', how='left')
coin_day_fg = coin_day.merge(fg_small, left_on='date_utc', right_on='fg_date_utc', how='left')

# Quick checks
print("Account-day unmatched %:", acc_day_fg['classification'].isna().mean()*100)
print("Market-day unmatched %:", mkt_day_fg['classification'].isna().mean()*100)
print("Coin-day unmatched %:", coin_day_fg['classification'].isna().mean()*100)

mkt_day_fg.head(5)[['date_utc','pnl_net_sum','win_rate','value','classification','sentiment_binary']]


Account-day unmatched %: 0.0
Market-day unmatched %: 0.0
Coin-day unmatched %: 0.0


,date_utc,pnl_net_sum,win_rate,value,classification,sentiment_binary
0,2023-04-30,0.000000,0.000000,60,Greed,Greed
1,2023-12-04,-12.501455,0.000000,74,Greed,Greed
2,2023-12-13,-294.499239,0.333333,65,Greed,Greed
3,2023-12-14,33.479148,0.300000,72,Greed,Greed
4,2023-12-16,306.664836,0.636364,67,Greed,Greed


In [ ]:
# Market stats by binary sentiment
mkt_stats = (
    mkt_day_fg.groupby('sentiment_binary', dropna=False)
    .agg(
        days=('date_utc','count'),
        avg_daily_net_pnl=('pnl_net_sum','mean'),
        median_daily_net_pnl=('pnl_net_sum','median'),
        avg_win_rate=('win_rate','mean'),
        avg_fg_value=('value','mean')
    )
    .reset_index()
    .sort_values('avg_daily_net_pnl', ascending=False)
)
mkt_stats


,sentiment_binary,days,avg_daily_net_pnl,median_daily_net_pnl,avg_win_rate,avg_fg_value
0,Fear,103,37121.306906,4057.187462,0.354723,30.669903
2,Neutral,64,19687.007152,1735.212143,0.318126,49.640625
1,Greed,309,16076.498880,1070.269622,0.401338,72.300971


In [ ]:
# Aggregate account-day by sentiment
acc_stats = (
    acc_day_fg.groupby(['account','sentiment_binary'], dropna=False)
    .agg(
        days=('date_utc','count'),
        net_pnl_sum=('pnl_net_sum','sum'),
        net_pnl_mean=('pnl_net_sum','mean'),
        win_rate_mean=('win_rate','mean'),
        size_usd_sum=('size_usd_sum','sum')
    )
    .reset_index()
)

# Pivot to align Fear vs. Greed per account
acc_pivot = acc_stats.pivot(index='account', columns='sentiment_binary', values='net_pnl_mean')
acc_pivot = acc_pivot.rename_axis(None, axis=1).reset_index()

# Compute differential: Greed - Fear
if {'Greed','Fear'}.issubset(acc_pivot.columns):
    acc_pivot['diff_greed_minus_fear'] = acc_pivot['Greed'] - acc_pivot['Fear']

# Rank accounts by diff
acc_rank = acc_pivot.sort_values('diff_greed_minus_fear', ascending=False)
acc_rank.head(15)


,account,Fear,Greed,Neutral,diff_greed_minus_fear
15,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,-2275.654564,64693.007602,-14.724355,66968.662166
9,0x430f09841d65beb3f27765503d0f850b8bce7713,461.880980,24496.614488,20112.790612,24034.733508
31,0xbee1707d6b44d4d52bfe19e41f8a828645437aab,1491.522565,14395.897149,861.176055,12904.374584
27,0xb1231a4a2dd02f2276fa3c5e2a2f3436e6bfed23,889.423737,11695.898590,7324.300128,10806.474853
25,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,1068.331778,9791.706008,NaN,8723.374230
22,0x92f17e8d81a944691c10e753af1b1baae1a2cd0d,784.209913,9381.757917,-447.569748,8597.548004
24,0xa520ded057a32086c40e7dd6ed4eb8efb82c00e0,-1113.863069,4526.605665,-26.878958,5640.468734
5,0x3998f134d6aaa2b6a5f723806d00fd2bbbbce891,-3835.023291,1191.582060,-205.285460,5026.605352
14,0x6d6a4b953f202f8df5bed40692e7fd865318264a,188.220865,5167.124584,312.312714,4978.903720
20,0x8381e6d82f1affd39a336e143e081ef7620a3b7f,-722.397279,1878.826850,907.112714,2601.224129


In [ ]:
# Minimum active days per sentiment regime to be included
MIN_DAYS = 5

# Compute days per account per regime
days_per_regime = (
    acc_day_fg.groupby(['account','sentiment_binary'])
    .agg(days=('date_utc','count'))
    .reset_index()
)

# Keep accounts that meet MIN_DAYS in either Fear or Greed (separately for each regime)
valid = days_per_regime[days_per_regime['days'] >= MIN_DAYS]

# Merge back to acc_stats to filter
acc_stats = (
    acc_day_fg.groupby(['account','sentiment_binary'], dropna=False)
    .agg(
        days=('date_utc','count'),
        net_pnl_sum=('pnl_net_sum','sum'),
        net_pnl_mean=('pnl_net_sum','mean'),
        win_rate_mean=('win_rate','mean'),
        size_usd_sum=('size_usd_sum','sum')
    )
    .reset_index()
)

acc_stats_valid = acc_stats.merge(valid[['account','sentiment_binary']], on=['account','sentiment_binary'], how='inner')

# Top and bottom accounts by net_pnl_mean for Fear and Greed
def top_bottom(df, regime, k=10):
    sub = df[df['sentiment_binary'] == regime].sort_values('net_pnl_mean', ascending=False)
    return sub.head(k), sub.tail(k)

top_fear, bottom_fear = top_bottom(acc_stats_valid, 'Fear', k=10)
top_greed, bottom_greed = top_bottom(acc_stats_valid, 'Greed', k=10)

print("Top Greed performers (mean net PnL):")
display(top_greed[['account','days','net_pnl_mean','win_rate_mean','size_usd_sum']])

print("Bottom Greed performers (mean net PnL):")
display(bottom_greed[['account','days','net_pnl_mean','win_rate_mean','size_usd_sum']])

print("Top Fear performers (mean net PnL):")
display(top_fear[['account','days','net_pnl_mean','win_rate_mean','size_usd_sum']])

print("Bottom Fear performers (mean net PnL):")
display(bottom_fear[['account','days','net_pnl_mean','win_rate_mean','size_usd_sum']])


Top Greed performers (mean net PnL):


,account,days,net_pnl_mean,win_rate_mean,size_usd_sum
41,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,7,64693.007602,0.330827,6819596.42
1,0x083384f897ee0f19899168e3b1bec365f52a9012,12,31361.024820,0.160587,31724382.45
24,0x430f09841d65beb3f27765503d0f850b8bce7713,11,24496.614488,0.363636,916512.91
79,0xbee1707d6b44d4d52bfe19e41f8a828645437aab,50,14395.897149,0.399742,34208983.37
36,0x513b8629fe877bb581bf244e326a047b249c4ff1,10,12544.366108,0.132976,97287813.60
70,0xb1231a4a2dd02f2276fa3c5e2a2f3436e6bfed23,153,11695.898590,0.341120,41419226.11
66,0xae5eacaf9c6b9111fd53034a602c192a04e082ed,6,9791.706008,0.413863,1067016.77
59,0x92f17e8d81a944691c10e753af1b1baae1a2cd0d,13,9381.757917,0.389229,6852968.52
30,0x4acb90e786d897ecffb614dc822eb231b4ffb9f4,19,6210.848745,0.420667,7163903.22
39,0x6d6a4b953f202f8df5bed40692e7fd865318264a,20,5167.124584,0.471520,238959.77


Bottom Greed performers (mean net PnL):


,account,days,net_pnl_mean,win_rate_mean,size_usd_sum
27,0x47add9a56df66b524d5e2c1993a43cde53b6ed85,73,1402.729197,0.353696,3400863.47
14,0x3998f134d6aaa2b6a5f723806d00fd2bbbbce891,23,1191.582060,0.204348,605757.33
17,0x39cef799f8b69da1995852eea189df24eb5cae3c,22,1128.075801,0.353794,1280082.73
6,0x271b280974205ca63b716753467d5a371de622ab,5,779.401395,0.422881,4546761.31
33,0x4f93fead39b70a1824f981a54d4e55b278e9f760,204,636.419822,0.261178,83171835.42
49,0x7f4f299f74eec87806a830e3caa9afa5f2b9db8f,14,572.691100,0.464015,348369.08
57,0x8477e447846c758f5a675856001ea72298fd9cb5,97,419.688896,0.332518,8126364.09
68,0xaf40fdc468c30116bd3307bcbf4a451a7ebf1deb,5,10.889729,0.600000,17730.20
76,0xbd5fead7180a9c139fa51a103cb6a2ce86ddb5c3,27,-2904.319417,0.162956,4199646.92
51,0x8170715b3b381dffb7062c0298972d4727a0a63b,25,-15070.965976,0.289947,3060724.85


Top Fear performers (mean net PnL):


,account,days,net_pnl_mean,win_rate_mean,size_usd_sum
0,0x083384f897ee0f19899168e3b1bec365f52a9012,7,145188.230721,0.532922,2.120961e+07
74,0xbaaaf6571ab7d571043ff1e313a9609a10637864,21,43374.630713,0.493746,5.636865e+07
20,0x420ab45e0bd8863569a5efbb9c05d91f40624641,7,18844.029611,0.578454,8.095100e+05
42,0x72c6a4624e1dffa724e6d00d64ceae698af892a0,19,17829.844712,0.460176,1.234632e+06
29,0x4acb90e786d897ecffb614dc822eb231b4ffb9f4,28,16025.812379,0.386563,2.758387e+07
35,0x513b8629fe877bb581bf244e326a047b249c4ff1,22,13527.546388,0.361176,2.442131e+08
75,0xbd5fead7180a9c139fa51a103cb6a2ce86ddb5c3,32,8357.594054,0.316234,1.451160e+07
5,0x271b280974205ca63b716753467d5a371de622ab,5,8346.163612,0.503636,1.837882e+07
50,0x8170715b3b381dffb7062c0298972d4727a0a63b,47,3225.623088,0.393151,4.976244e+06
45,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,32,2280.734191,0.771385,4.369541e+06


Bottom Fear performers (mean net PnL):


,account,days,net_pnl_mean,win_rate_mean,size_usd_sum
56,0x8477e447846c758f5a675856001ea72298fd9cb5,54,170.540977,0.297491,1.888370e+06
2,0x23e7a7f8d14b550961925fbfdaa92f5d195ba5bd,20,15.828758,0.397524,2.857364e+06
72,0xb899e522b5715391ae1d4f137653e7906c5e2115,28,-111.657749,0.415252,1.028540e+08
26,0x47add9a56df66b524d5e2c1993a43cde53b6ed85,50,-125.412258,0.351002,7.098607e+05
7,0x28736f43f1e871e6aa8b1148d38d4994275d72c4,54,-220.773254,0.424576,1.407919e+06
53,0x8381e6d82f1affd39a336e143e081ef7620a3b7f,7,-722.397279,0.276398,3.728168e+05
16,0x39cef799f8b69da1995852eea189df24eb5cae3c,20,-973.750547,0.373722,1.565056e+07
63,0xa520ded057a32086c40e7dd6ed4eb8efb82c00e0,16,-1113.863069,0.125000,3.151002e+05
40,0x72743ae2822edd658c0c50608fd7c5c501b2afbd,11,-2275.654564,0.149828,4.507660e+06
13,0x3998f134d6aaa2b6a5f723806d00fd2bbbbce891,15,-3835.023291,0.156217,7.642852e+05


In [ ]:
mkt_summary = (
    mkt_day_fg.groupby('sentiment_binary', dropna=False)
    .agg(
        days=('date_utc','count'),
        avg_daily_net_pnl=('pnl_net_sum','mean'),
        med_daily_net_pnl=('pnl_net_sum','median'),
        avg_win_rate=('win_rate','mean'),
        avg_trade_count=('trades_count','mean'),
        avg_size_usd=('size_usd_sum','mean'),
        avg_fg_value=('value','mean')
    )
    .reset_index()
    .sort_values('avg_daily_net_pnl', ascending=False)
)
mkt_summary


,sentiment_binary,days,avg_daily_net_pnl,med_daily_net_pnl,avg_win_rate,avg_trade_count,avg_size_usd,avg_fg_value
0,Fear,103,37121.306906,4057.187462,0.354723,804.009709,5.852184e+06,30.669903
2,Neutral,64,19687.007152,1735.212143,0.318126,618.171875,2.995964e+06,49.640625
1,Greed,309,16076.498880,1070.269622,0.401338,287.533981,1.283724e+06,72.300971


In [ ]:
# Aggregate coin-day with sentiment
coin_day_fg = (
    coin_day.merge(mkt_day_fg[['date_utc','classification','sentiment_binary','value']], on='date_utc', how='left')
)

# Require a minimum number of coin-days per regime to avoid noise
MIN_COIN_DAYS = 10

coin_stats = (
    coin_day_fg.groupby(['coin','sentiment_binary'], dropna=False)
    .agg(
        days=('date_utc','count'),
        net_pnl_sum=('pnl_net_sum','sum'),
        net_pnl_mean=('pnl_net_sum','mean'),
        win_rate_mean=('win_rate','mean'),
        size_usd_sum=('size_usd_sum','sum')
    )
    .reset_index()
)

coin_stats_valid = coin_stats[coin_stats['days'] >= MIN_COIN_DAYS]

# Top/bottom by net_pnl_mean within each regime
def top_bottom_coins(df, regime, k=10):
    sub = df[df['sentiment_binary'] == regime].sort_values('net_pnl_mean', ascending=False)
    return sub.head(k), sub.tail(k)

top_greed_coins, bottom_greed_coins = top_bottom_coins(coin_stats_valid, 'Greed', k=10)
top_fear_coins, bottom_fear_coins   = top_bottom_coins(coin_stats_valid, 'Fear',  k=10)

print("Top Greed coins:")
display(top_greed_coins[['coin','days','net_pnl_mean','win_rate_mean','size_usd_sum']])

print("Bottom Greed coins:")
display(bottom_greed_coins[['coin','days','net_pnl_mean','win_rate_mean','size_usd_sum']])

print("Top Fear coins:")
display(top_fear_coins[['coin','days','net_pnl_mean','win_rate_mean','size_usd_sum']])

print("Bottom Fear coins:")
display(bottom_fear_coins[['coin','days','net_pnl_mean','win_rate_mean','size_usd_sum']])


Top Greed coins:


,coin,days,net_pnl_mean,win_rate_mean,size_usd_sum
7,@107,74,36616.827697,0.453331,37446836.13
466,ZRO,14,10287.796987,0.576574,887416.69
390,SOL,91,6646.362165,0.340894,26978037.60
251,HYPE,65,5198.623014,0.358392,21591758.03
302,MELANIA,17,3202.831886,0.171881,1913094.14
430,USUAL,26,2679.119351,0.541736,806339.27
113,@85,10,2649.433983,0.400000,41960.48
203,EIGEN,23,2253.307947,0.372941,305233.26
215,ETHFI,14,2164.479600,0.357143,479816.30
140,ANIME,10,2118.467774,0.483037,583411.55


Bottom Greed coins:


,coin,days,net_pnl_mean,win_rate_mean,size_usd_sum
452,XRP,70,8.619499,0.421340,506796.59
196,DOT,17,8.579500,0.557890,153547.61
463,ZK,14,7.030458,0.543651,3515.14
281,LDO,29,6.255824,0.354502,23626.54
153,ATOM,21,4.073870,0.359606,7665.85
386,SEI,12,-83.201294,0.343750,332168.15
395,STRK,13,-117.498403,0.430769,83026.40
353,POPCAT,16,-386.116899,0.357187,265542.08
260,IO,23,-957.161159,0.370973,251245.00
416,TRUMP,13,-33330.556943,0.403451,3556830.78


Top Fear coins:


,coin,days,net_pnl_mean,win_rate_mean,size_usd_sum
250,HYPE,54,25028.088200,0.414881,9.004391e+07
389,SOL,53,16175.821189,0.334135,7.847738e+07
205,ENA,14,11285.625483,0.311998,3.835742e+05
211,ETH,74,9792.701961,0.358378,3.719197e+07
178,BTC,68,7023.528918,0.333614,3.561035e+08
301,MELANIA,42,6690.716556,0.454071,4.277487e+06
415,TRUMP,16,3135.352249,0.397130,2.419068e+06
399,SUI,43,1637.801238,0.422167,2.472473e+06
265,JELLY,10,1553.580831,0.829024,9.570542e+05
133,AIXBT,11,932.185613,0.405437,3.910585e+05


Bottom Fear coins:


,coin,days,net_pnl_mean,win_rate_mean,size_usd_sum
411,TNSR,13,6.422405,0.294872,7905.40
198,DYDX,10,-7.163388,0.205000,362100.19
107,@8,14,-12.190044,0.051020,32021.96
72,@4,20,-18.290858,0.322630,58270.71
170,BNB,14,-25.130639,0.458874,161847.21
360,PURR/USDC,28,-193.066648,0.074405,159308.93
271,JUP,13,-258.402142,0.351282,29490.85
124,ADA,12,-2474.458963,0.233562,941524.22
6,@107,50,-2901.115177,0.309367,9924424.08
217,FARTCOIN,39,-3833.830688,0.339957,2106146.43


In [ ]:
# Pick top/bottom coins per regime using thresholds
TOP_K = 10
MIN_DAYS = 10

# Helper to get sets
def coin_sets(df, regime):
    sub = df[(df['sentiment_binary']==regime) & (df['days']>=MIN_DAYS)]
    top = sub.sort_values('net_pnl_mean', ascending=False).head(TOP_K)['coin']
    bottom = sub.sort_values('net_pnl_mean', ascending=True).head(TOP_K)['coin']
    return set(top), set(bottom)

top_greed_set, bottom_greed_set = coin_sets(coin_stats_valid, 'Greed')
top_fear_set,  bottom_fear_set  = coin_sets(coin_stats_valid, 'Fear')

rules = {
    'Greed': {
        'favor_coins': sorted(list(top_greed_set)),
        'avoid_coins': sorted(list(bottom_greed_set)),
    },
    'Fear': {
        'favor_coins': sorted(list(top_fear_set)),
        'avoid_coins': sorted(list(bottom_fear_set)),
    }
}

rules


{'Greed': {'favor_coins': ['@107',
   '@85',
   'ANIME',
   'EIGEN',
   'ETHFI',
   'HYPE',
   'MELANIA',
   'SOL',
   'USUAL',
   'ZRO'],
  'avoid_coins': ['ATOM',
   'DOT',
   'IO',
   'LDO',
   'POPCAT',
   'SEI',
   'STRK',
   'TRUMP',
   'XRP',
   'ZK']},
 'Fear': {'favor_coins': ['AIXBT',
   'BTC',
   'ENA',
   'ETH',
   'HYPE',
   'JELLY',
   'MELANIA',
   'SOL',
   'SUI',
   'TRUMP'],
  'avoid_coins': ['@107',
   '@4',
   '@8',
   'ADA',
   'BNB',
   'DYDX',
   'FARTCOIN',
   'JUP',
   'PURR/USDC',
   'TNSR']}}

In [ ]:
from pathlib import Path

OUT_DIR = Path('/content/drive/MyDrive/Crypto/processed')
OUT_DIR.mkdir(parents=True, exist_ok=True)

acc_day_fg.to_csv(OUT_DIR / 'account_day_with_fg.csv', index=False)
mkt_day_fg.to_csv(OUT_DIR / 'market_day_with_fg.csv', index=False)
coin_day_fg.to_csv(OUT_DIR / 'coin_day_with_fg.csv', index=False)

print("Saved to:", OUT_DIR)


Saved to: /content/drive/MyDrive/Crypto/processed


In [ ]:
# Map each coin-day to favor/avoid/neutral based on regime
def label_coin_policy(row):
    regime = row['sentiment_binary']
    c = row['coin']
    if regime == 'Greed':
        if c in rules['Greed']['favor_coins']: return 'favor'
        if c in rules['Greed']['avoid_coins']: return 'avoid'
    if regime == 'Fear':
        if c in rules['Fear']['favor_coins']: return 'favor'
        if c in rules['Fear']['avoid_coins']: return 'avoid'
    return 'neutral'

coin_day_fg['policy'] = coin_day_fg.apply(label_coin_policy, axis=1)

# Aggregate performance by regime and policy
policy_stats = (
    coin_day_fg.groupby(['sentiment_binary','policy'])
    .agg(
        days=('date_utc','count'),
        net_pnl_mean=('pnl_net_sum','mean'),
        net_pnl_sum=('pnl_net_sum','sum'),
        win_rate=('win_rate','mean'),
        size_usd_sum=('size_usd_sum','sum')
    )
    .reset_index()
    .sort_values(['sentiment_binary','policy'])
)

policy_stats


,sentiment_binary,policy,days,net_pnl_mean,net_pnl_sum,win_rate,size_usd_sum
0,Fear,avoid,213,-1567.653552,-3.339102e+05,0.270675,1.378304e+07
1,Fear,favor,385,10380.480490,3.996485e+06,0.389649,5.727174e+08
2,Fear,neutral,645,249.488107,1.609198e+05,0.355687,1.627445e+07
3,Greed,avoid,228,-2030.267978,-4.629011e+05,0.410696,5.183964e+06
4,Greed,favor,344,11774.177094,4.050317e+06,0.393461,9.103390e+07
5,Greed,neutral,2535,544.466403,1.380222e+06,0.436525,3.004529e+08
6,Neutral,neutral,705,1787.189302,1.259968e+06,0.382788,1.917417e+08


In [ ]:
# Prepare coin-day with a weight per policy
weight_map = {'favor': 1.0, 'neutral': 0.5, 'avoid': 0.0}
coin_day_fg['weight'] = coin_day_fg['policy'].map(weight_map).fillna(0.5)

# Compute daily baseline and weighted PnL (sum across coins per day)
daily_baseline = (
    coin_day_fg.groupby('date_utc', as_index=False)
    .agg(baseline_pnl=('pnl_net_sum','sum'))
    .merge(mkt_day_fg[['date_utc','sentiment_binary','value']], on='date_utc', how='left')
)

daily_weighted = (
    coin_day_fg.assign(weighted_pnl=lambda d: d['pnl_net_sum'] * d['weight'])
    .groupby('date_utc', as_index=False)['weighted_pnl'].sum()
)

# Join and compute improvements
daily_compare = (
    daily_baseline.merge(daily_weighted, on='date_utc', how='left')
    .rename(columns={'weighted_pnl':'strategy_pnl'})
)
daily_compare['improvement'] = daily_compare['strategy_pnl'] - daily_compare['baseline_pnl']

# Summary stats by regime
compare_stats = (
    daily_compare.groupby('sentiment_binary', dropna=False)
    .agg(
        days=('date_utc','count'),
        baseline_mean=('baseline_pnl','mean'),
        strategy_mean=('strategy_pnl','mean'),
        improvement_mean=('improvement','mean'),
        improvement_sum=('improvement','sum')
    )
    .reset_index()
    .sort_values('strategy_mean', ascending=False)
)

compare_stats.head(10)


,sentiment_binary,days,baseline_mean,strategy_mean,improvement_mean,improvement_sum
0,Fear,103,37121.306906,39581.989352,2460.682447,253450.292011
1,Greed,309,16076.498880,15341.191219,-735.307661,-227210.067269
2,Neutral,64,19687.007152,9843.503576,-9843.503576,-629984.228873


In [ ]:
# Regime-specific weights
regime_weights = {
    'Greed':  {'favor': 1.0,  'neutral': 0.25, 'avoid': 0.0},
    'Fear':   {'favor': 1.0,  'neutral': 0.50, 'avoid': 0.0},
    'Neutral':{'favor': 0.75, 'neutral': 0.25, 'avoid': 0.0},
}

def row_weight(r):
    rw = regime_weights.get(r['sentiment_binary'], {'favor': 0.5, 'neutral': 0.25, 'avoid': 0.0})
    return rw.get(r['policy'], 0.25)

coin_day_fg['weight2'] = coin_day_fg.apply(row_weight, axis=1)

daily_weighted2 = (
    coin_day_fg.assign(weighted_pnl=lambda d: d['pnl_net_sum'] * d['weight2'])
    .groupby('date_utc', as_index=False)['weighted_pnl'].sum()
    .rename(columns={'weighted_pnl':'strategy_pnl_w2'})
)

daily_compare2 = (
    daily_baseline.merge(daily_weighted2, on='date_utc', how='left')
)
daily_compare2['improvement_w2'] = daily_compare2['strategy_pnl_w2'] - daily_compare2['baseline_pnl']

compare_stats2 = (
    daily_compare2.groupby('sentiment_binary', dropna=False)
    .agg(
        days=('date_utc','count'),
        baseline_mean=('baseline_pnl','mean'),
        strategy_mean=('strategy_pnl_w2','mean'),
        improvement_mean=('improvement_w2','mean'),
        improvement_sum=('improvement_w2','sum')
    )
    .reset_index()
    .sort_values('strategy_mean', ascending=False)
)

compare_stats2


,sentiment_binary,days,baseline_mean,strategy_mean,improvement_mean,improvement_sum
0,Fear,103,37121.306906,39581.989352,2460.682447,253450.292011
1,Greed,309,16076.498880,14224.506484,-1851.992396,-572265.650408
2,Neutral,64,19687.007152,4921.751788,-14765.255364,-944976.343309


In [ ]:
# Updated regime weights:
# - Greed: favor=1.0, neutral=0.0, avoid=0.0
# - Fear:  favor=1.0, neutral=0.5, avoid=0.0 (unchanged)
# - Neutral: favor=0.75, neutral=0.125, avoid=0.0 (halved neutral)
regime_weights_v3 = {
    'Greed':  {'favor': 1.0,  'neutral': 0.0,  'avoid': 0.0},
    'Fear':   {'favor': 1.0,  'neutral': 0.50, 'avoid': 0.0},
    'Neutral':{'favor': 0.75, 'neutral': 0.125,'avoid': 0.0},
}

def row_weight_v3(r):
    rw = regime_weights_v3.get(r['sentiment_binary'], {'favor': 0.5, 'neutral': 0.25, 'avoid': 0.0})
    return rw.get(r['policy'], 0.25)

coin_day_fg['weight_v3'] = coin_day_fg.apply(row_weight_v3, axis=1)

daily_weighted_v3 = (
    coin_day_fg.assign(weighted_pnl=lambda d: d['pnl_net_sum'] * d['weight_v3'])
    .groupby('date_utc', as_index=False)['weighted_pnl'].sum()
    .rename(columns={'weighted_pnl':'strategy_pnl_v3'})
)

daily_compare_v3 = (
    daily_baseline.merge(daily_weighted_v3, on='date_utc', how='left')
)
daily_compare_v3['improvement_v3'] = daily_compare_v3['strategy_pnl_v3'] - daily_compare_v3['baseline_pnl']

compare_stats_v3 = (
    daily_compare_v3.groupby('sentiment_binary', dropna=False)
    .agg(
        days=('date_utc','count'),
        baseline_mean=('baseline_pnl','mean'),
        strategy_mean=('strategy_pnl_v3','mean'),
        improvement_mean=('improvement_v3','mean'),
        improvement_sum=('improvement_v3','sum')
    )
    .reset_index()
    .sort_values('strategy_mean', ascending=False)
)

compare_stats_v3


,sentiment_binary,days,baseline_mean,strategy_mean,improvement_mean,improvement_sum
0,Fear,103,37121.306906,39581.989352,2460.682447,2.534503e+05
1,Greed,309,16076.498880,13107.821749,-2968.677131,-9.173212e+05
2,Neutral,64,19687.007152,2460.875894,-17226.131258,-1.102472e+06


In [ ]:
# Step 18: Stricter Greed favor set, updated policy labels

# 1) Rebuild Greed favor set with stricter thresholds
# Criteria: days ≥ 30 in Greed, win_rate_mean ≥ 0.45, net_pnl_mean ≥ 0
greed_quality = (
    coin_day_fg[coin_day_fg['sentiment_binary']=='Greed']
    .groupby('coin')
    .agg(
        days=('date_utc','count'),
        win_rate_mean=('win_rate','mean'),
        net_pnl_mean=('pnl_net_sum','mean'),
        size_usd_sum=('size_usd_sum','sum')
    )
    .reset_index()
)

greed_favor_q = greed_quality[
    (greed_quality['days'] >= 30) &
    (greed_quality['win_rate_mean'] >= 0.45) &
    (greed_quality['net_pnl_mean'] >= 0)
]['coin'].tolist()

print("Greed favor (strict):", sorted(greed_favor_q))

# add a liquidity filter (top 50% by size within Greed)
use_liquidity_filter = True
if use_liquidity_filter and len(greed_favor_q) > 0:
    median_liq = greed_quality['size_usd_sum'].median()
    greed_favor_q = [
        c for c in greed_favor_q
        if greed_quality.loc[greed_quality['coin']==c, 'size_usd_sum'].iloc[0] >= median_liq
    ]
    print("Greed favor (strict + liquid):", sorted(greed_favor_q))

# 2) Update rules and policy labeling
rules_q = {
    'Greed': {
        'favor_coins': set(greed_favor_q),
        'avoid_coins': set(rules['Greed']['avoid_coins'])
    },
    'Fear': {
        'favor_coins': set(rules['Fear']['favor_coins']),
        'avoid_coins': set(rules['Fear']['avoid_coins'])
    }
}

def label_coin_policy_q(row):
    regime, c = row['sentiment_binary'], row['coin']
    if regime == 'Greed':
        if c in rules_q['Greed']['favor_coins']: return 'favor'
        if c in rules_q['Greed']['avoid_coins']: return 'avoid'
        return 'neutral'
    if regime == 'Fear':
        if c in rules_q['Fear']['favor_coins']: return 'favor'
        if c in rules_q['Fear']['avoid_coins']: return 'avoid'
        return 'neutral'
    return 'neutral'

coin_day_fg['policy_q'] = coin_day_fg.apply(label_coin_policy_q, axis=1)

# 3) Quick sanity check of policy distribution
policy_counts = (
    coin_day_fg.groupby(['sentiment_binary','policy_q'])
    .size()
    .reset_index(name='rows')
    .sort_values(['sentiment_binary','rows'], ascending=[True, False])
)
policy_counts


Greed favor (strict): ['@107', 'DOGE', 'HBAR', 'LTC', 'PENGU', 'PURR', 'RUNE', 'SUI', 'kBONK', 'kPEPE']
Greed favor (strict + liquid): ['@107', 'DOGE', 'HBAR', 'LTC', 'PENGU', 'PURR', 'RUNE', 'SUI', 'kBONK', 'kPEPE']


,sentiment_binary,policy_q,rows
2,Fear,neutral,645
1,Fear,favor,385
0,Fear,avoid,213
5,Greed,neutral,2374
4,Greed,favor,505
3,Greed,avoid,228
6,Neutral,neutral,705


In [ ]:
#  Regime weights + robust per-day liquidity guard (Greed off, Neutral off, Fear on)

import pandas as pd

#  Regime weights
# - Greed: off
# - Neutral: off
# - Fear: favor strong, neutral half, avoid none
regime_weights_v5 = {
    'Greed':  {'favor': 0.0,  'neutral': 0.0,  'avoid': 0.0},  # Greed OFF
    'Fear':   {'favor': 1.0,  'neutral': 0.50, 'avoid': 0.0},  # Fear ON (kept beneficial setting)
    'Neutral':{'favor': 0.0,  'neutral': 0.0,  'avoid': 0.0},  # Neutral OFF
}

def row_weight_v5(r):
    rw = regime_weights_v5.get(r['sentiment_binary'], {'favor': 0.0, 'neutral': 0.0, 'avoid': 0.0})
    return rw.get(r['policy_q'], 0.0)

#Build day sentiment map and per-day liquidity median for Greed days
day_mode = (
    coin_day_fg.groupby('date_utc', as_index=False)
    .agg(day_sentiment=('sentiment_binary','first'))
)

greed_days = day_mode.loc[day_mode['day_sentiment']=='Greed', 'date_utc']
per_day_median = (
    coin_day_fg[coin_day_fg['date_utc'].isin(greed_days)]
    .groupby('date_utc', as_index=False)['size_usd_sum'].median()
    .rename(columns={'size_usd_sum':'day_median_size'})
)

# Merge back sentiment and day medians; apply guard only on Greed days
coin_day_guarded = (
    coin_day_fg.merge(day_mode, on='date_utc', how='left')
               .merge(per_day_median, on='date_utc', how='left')
)

mask_greed = coin_day_guarded['day_sentiment'].eq('Greed')
mask_keep = (~mask_greed) | (coin_day_guarded['size_usd_sum'] >= coin_day_guarded['day_median_size'].fillna(-1))
coin_day_guarded = coin_day_guarded.loc[mask_keep].copy()

#  Weights, daily strategy PnL
coin_day_guarded['weight_v5'] = coin_day_guarded.apply(row_weight_v5, axis=1)
coin_day_guarded['strategy_contrib'] = coin_day_guarded['pnl_net_sum'] * coin_day_guarded['weight_v5']

daily_weighted_v5 = (
    coin_day_guarded.groupby('date_utc', as_index=False)['strategy_contrib'].sum()
    .rename(columns={'strategy_contrib':'strategy_pnl_v5'})
)


# daily_baseline = (
#     coin_day_fg.groupby('date_utc', as_index=False)
#     .agg(baseline_pnl=('pnl_net_sum','sum'))
#     .merge(mkt_day_fg[['date_utc','sentiment_binary','value']], on='date_utc', how='left')
# )

# 6) Compare by regime
daily_compare_v5 = daily_baseline.merge(daily_weighted_v5, on='date_utc', how='left')
daily_compare_v5['improvement_v5'] = daily_compare_v5['strategy_pnl_v5'] - daily_compare_v5['baseline_pnl']

compare_stats_v5 = (
    daily_compare_v5.groupby('sentiment_binary', dropna=False)
    .agg(
        days=('date_utc','count'),
        baseline_mean=('baseline_pnl','mean'),
        strategy_mean=('strategy_pnl_v5','mean'),
        improvement_mean=('improvement_v5','mean'),
        improvement_sum=('improvement_v5','sum')
    )
    .reset_index()
    .sort_values('strategy_mean', ascending=False)
)

compare_stats_v5


,sentiment_binary,days,baseline_mean,strategy_mean,improvement_mean,improvement_sum
0,Fear,103,37121.306906,39581.989352,2460.682447,2.534503e+05
1,Greed,309,16076.498880,0.000000,-16076.498880,-4.967638e+06
2,Neutral,64,19687.007152,0.000000,-19687.007152,-1.259968e+06


In [ ]:
import json
from pathlib import Path

OUT_DIR = Path('/content/drive/MyDrive/Crypto/processed')
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Save comparison table and per-day series
daily_compare_v5.to_csv(OUT_DIR / 'daily_compare_v1.csv', index=False)
compare_stats_v5.to_csv(OUT_DIR / 'regime_compare_stats_v1.csv', index=False)

#  Save the final policy (favor/avoid lists and weights)
final_policy = {
    'rules': {
        'Greed': {
            'favor_coins': [],  # off
            'avoid_coins': sorted(list(rules['Greed']['avoid_coins'])),
        },
        'Fear': {
            'favor_coins': sorted(list(rules['Fear']['favor_coins'])),
            'avoid_coins': sorted(list(rules['Fear']['avoid_coins'])),
        },
        'Neutral': {
            'favor_coins': [],
            'avoid_coins': [],
        }
    },
    'weights': {
        'Greed':  {'favor': 0.0,  'neutral': 0.0,  'avoid': 0.0},
        'Fear':   {'favor': 1.0,  'neutral': 0.50, 'avoid': 0.0},
        'Neutral':{'favor': 0.0,  'neutral': 0.0,  'avoid': 0.0},
    },
    'liquidity_guard': {
        'enabled_on_greed': True,
        'method': 'per-day median size_usd_sum; keep >= median'
    }
}

with open(OUT_DIR / 'policy_v1.json', 'w') as f:
    json.dump(final_policy, f, indent=2)

print("Saved:", OUT_DIR)


Saved: /content/drive/MyDrive/Crypto/processed


In [ ]:
print("Regime summary (v1):")
display(compare_stats_v5)

overall = {
    'baseline_mean_total': daily_compare_v5['baseline_pnl'].mean(),
    'strategy_mean_total': daily_compare_v5['strategy_pnl_v5'].mean(),
    'improvement_mean_total': (daily_compare_v5['strategy_pnl_v5'] - daily_compare_v5['baseline_pnl']).mean(),
    'improvement_sum_total': (daily_compare_v5['strategy_pnl_v5'] - daily_compare_v5['baseline_pnl']).sum(),
}
overall


Regime summary (v1):


,sentiment_binary,days,baseline_mean,strategy_mean,improvement_mean,improvement_sum
0,Fear,103,37121.306906,39581.989352,2460.682447,2.534503e+05
1,Greed,309,16076.498880,0.000000,-16076.498880,-4.967638e+06
2,Neutral,64,19687.007152,0.000000,-19687.007152,-1.259968e+06


{'baseline_mean_total': np.float64(21115.758871576196),
 'strategy_mean_total': np.float64(8565.010301048453),
 'improvement_mean_total': np.float64(-12550.748570527745),
 'improvement_sum_total': np.float64(-5974156.319571206)}

In [ ]:
from pathlib import Path
p = Path('/content/drive/MyDrive/Crypto/processed')
list(p.glob('*v1*')), list(p.glob('policy_v1.json'))


([PosixPath('/content/drive/MyDrive/Crypto/processed/daily_compare_v1.csv'),
  PosixPath('/content/drive/MyDrive/Crypto/processed/regime_compare_stats_v1.csv'),
  PosixPath('/content/drive/MyDrive/Crypto/processed/policy_v1.json')],
 [PosixPath('/content/drive/MyDrive/Crypto/processed/policy_v1.json')])

In [ ]:
import pandas as pd
pd.read_csv('/content/drive/MyDrive/Crypto/processed/daily_compare_v1.csv').head()
pd.read_csv('/content/drive/MyDrive/Crypto/processed/regime_compare_stats_v1.csv').head()

import json, pprint
with open('/content/drive/MyDrive/Crypto/processed/policy_v1.json') as f:
    pprint.pp(json.load(f))


{'rules': {'Greed': {'favor_coins': [],
                     'avoid_coins': ['ATOM',
                                     'DOT',
                                     'IO',
                                     'LDO',
                                     'POPCAT',
                                     'SEI',
                                     'STRK',
                                     'TRUMP',
                                     'XRP',
                                     'ZK']},
           'Fear': {'favor_coins': ['AIXBT',
                                    'BTC',
                                    'ENA',
                                    'ETH',
                                    'HYPE',
                                    'JELLY',
                                    'MELANIA',
                                    'SOL',
                                    'SUI',
                                    'TRUMP'],
                    'avoid_coins': ['@107',
                           